Step 1. Build a CaF₂ Cluster with ASE

In [1]:
from ase import Atoms
from ase.build import make_supercell
from ase.io import write
from ase.build import bulk
from ase.cluster import wulff_construction
from ase.io import write

# Build CaF2 fluorite unit cell
caf2 = bulk('CaF2', 'fluorite', a=5.46)  # experimental lattice ~5.46 Å

# Cut out a small cluster (~1 nm) with CaF2 stoichiometry
# (Alternative: just take supercell and slice central atoms)
cluster = caf2.repeat((2,2,2))  # 2x2x2 supercell
cluster.center(vacuum=6.0)

write("caf2_cluster.xyz", cluster)
print("Cluster with", len(cluster), "atoms written to caf2_cluster.xyz")

Cluster with 24 atoms written to caf2_cluster.xyz


Step 2. Build a CaF₂:Er Cluster with ASE

In [2]:
from ase.build import bulk
from ase.io import write
import numpy as np

# Step 1: Build CaF2 fluorite cell
caf2 = bulk('CaF2', 'fluorite', a=5.46)

# Step 2: Make a small cluster (supercell with vacuum)
cluster = caf2.repeat((2,2,2))
cluster.center(vacuum=6.0)

# Step 3: Substitute one Ca -> Er to get Ca3:Er ratio in cluster
symbols = cluster.get_chemical_symbols()
ca_indices = [i for i, s in enumerate(symbols) if s == "Ca"]

if len(ca_indices) < 4:
    raise RuntimeError("Not enough Ca atoms to make Ca3:Er substitution!")

# Replace the 4th Ca with Er
symbols[ca_indices[3]] = "Er"
cluster.set_chemical_symbols(symbols)

# Step 4: Save
write("ca3f_er_cluster.xyz", cluster)
print("Cluster with", len(cluster), "atoms written to ca3f_er_cluster.xyz")

Cluster with 24 atoms written to ca3f_er_cluster.xyz


3. Simulate

In [ ]:
import os, sys, time, inspect, psutil
import numpy as np
from ase.io import read, write
from ase.optimize import LBFGS
from gpaw import GPAW, FermiDirac
from gpaw.tddft import TDDFT
from gpaw.mixer import Mixer, MixerSum, MixerDif
from gpaw.poisson import PoissonSolver
from gpaw import GPAW
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from gpaw.lrtddft import LrTDDFT


# --- DeltaKick ---
class DeltaKick:
    def __init__(self, strength=0.001, direction=(1, 0, 0)):
        self._strength = strength
        self.direction = np.array(direction, dtype=float)
        self.applied = False

    def strength(self, time):
        if not self.applied and abs(time) < 1e-8:
            self.applied = True
            return self._strength * self.direction
        return np.zeros(3)


# --- Step 1: Safer LCAO Relax ---
def relax_lcao(xyz_file):
    """
    Relax cluster geometry in LCAO mode with safer SCF settings.
    Uses gentle mixing and looser criteria to improve convergence.
    """
    name = os.path.splitext(xyz_file)[0]
    gpw_lcao = f"{name}_lcao.gpw"

    # Load system
    atoms = read(xyz_file)
    atoms.center(vacuum=10.0)

    # Calculator with cluster-friendly SCF settings
    calc = GPAW(mode="lcao",
                basis="dzp",
                xc="PBE",
                occupations=FermiDirac(0.1),   # slightly larger smearing
                kpts=(1, 1, 1),
                mixer={'backend': 'pulay',
                       'beta': 0.05,    # gentler mixing
                       'nmaxold': 5,    # fewer densities kept
                       'weight': 50.0}, # softer damping
                convergence={'energy': 1e-2, 'density': 1e-2},
                txt=f"{name}_lcao.log")
    atoms.calc = calc

    # Initial SCF to stabilize density before relaxation
    print("Running initial SCF to stabilize density...")
    atoms.get_potential_energy()

    # Geometry optimization (looser fmax first)
    dyn = LBFGS(atoms, logfile=f"{name}_lcao_opt.log")
    dyn.run(fmax=0.15, steps=200)   # relaxed force tolerance

    # Save results
    calc.write(gpw_lcao, mode="all")
    write(f"{name}_lcao_relaxed.xyz", atoms)
    print(f"✔ {name}: saved {gpw_lcao}")
    return gpw_lcao


# --- Step 2: FD Restart---
def groundstate_fd(gpw_lcao, virt_buffer=20, h=0.30):
    name = gpw_lcao.replace("_lcao.gpw", "_fd")
    gpw_fd = f"{name}.gpw"
    if os.path.exists(gpw_fd):
        print(f"Using cached {gpw_fd}")
        return gpw_fd

    lcao_calc = GPAW(gpw_lcao)
    n_occ = int(np.ceil(lcao_calc.get_number_of_electrons() / 2.0))
    n_bands = n_occ + virt_buffer

    calc = GPAW(gpw_lcao,
                mode="fd",
                h=h,
                xc="PBE",
                occupations=FermiDirac(0.03),
                nbands=n_bands,
                poissonsolver=PoissonSolver('fd', eps=1e-12),
                mixer={"beta": 0.05, "nmaxold": 10, "weight": 100},
                convergence={"energy": 3e-4, "density": 2e-4, "eigenstates": 4},
                symmetry={"point_group": False},
                txt=f"{name}.log")
    calc.get_potential_energy()
    calc.write(gpw_fd, mode="all")
    return gpw_fd

# -----------------------------------------------------
#  TDDFT Runner (Modern GPAW ≥ 25.x)
# -----------------------------------------------------
def run_lrtddft(gpw_fd, prefix="system", emax=10.0, sigma=0.1):
    """Linear-Response TDDFT with auto memory and auto state detection."""
    print(f"\n=== Running LrTDDFT on {gpw_fd} ===")
    start_time = time.time()

    calc = GPAW(gpw_fd)
    free_mem_gb = psutil.virtual_memory().available / 1e9
    total_states = len(calc.get_eigenvalues())
    print(f"Available RAM ≈ {free_mem_gb:.2f} GB | KS states = {total_states}")

    # --- Run modern LrTDDFT ---
    try:
        lr = LrTDDFT(calc)
        lr.diagonalize()
        print("TDDFT: modern auto-state solver active.")
    except Exception as e:
        print(f"⚠️ Fallback legacy solver due to: {e}")
        lr = LrTDDFT(calc)
        lr.diagonalize()

    # --- Extract excitations ---
    energies, osc = [], []
    for exc in lr:
        e = exc.get_energy() * 27.2114  # eV
        f = np.linalg.norm(exc.get_oscillator_strength())
        if e <= emax:
            energies.append(e)
            osc.append(f)
    print(f"Extracted {len(energies)} excitations below {emax:.1f} eV.")

    # --- Save raw data ---
    np.savetxt(f"{prefix}_sticks.csv",
               np.column_stack([energies, osc]),
               delimiter=",", header="Energy(eV),OscStrength", comments="")

    # --- Broaden and plot spectrum ---
    x = np.linspace(0, emax + 0.5, 1000)
    y = np.zeros_like(x)
    for e, f in zip(energies, osc):
        y[np.argmin(np.abs(x - e))] += f
    y_smooth = gaussian_filter1d(y, sigma * 100)

    plt.figure(figsize=(6,4))
    plt.plot(x, y_smooth, label=f"{prefix}")
    plt.scatter(energies, osc, c="red", s=15, label="Excitations")
    plt.xlabel("Energy (eV)")
    plt.ylabel("Oscillator strength (a.u.)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{prefix}_spectrum.png", dpi=300)
    plt.close()

    runtime = (time.time() - start_time)/60
    print(f"TDDFT runtime: {runtime:.2f} min | Saved: {prefix}_spectrum.png\n")

    return f"{prefix}_sticks.csv", f"{prefix}_spectrum.png"

# -----------------------------------------------------
#  Main Simulation Pipeline
# -----------------------------------------------------
def run_pipeline(xyz_file):
    prefix = xyz_file.replace(".xyz", "")
    gpw_lcao = f"{prefix}_lcao.gpw"
    gpw_fd   = f"{prefix}_fd.gpw"
    spec_csv = f"{prefix}_sticks.csv"
    spec_png = f"{prefix}_spectrum.png"

    print(f"\n=== Starting pipeline for {prefix} ===")

    # --- Step 1: Relax geometry (LCAO) ---
    if os.path.exists(gpw_lcao):
        print(f"✔ Cached LCAO found: {gpw_lcao}")
    else:
        print(f"Running LCAO relaxation for {prefix} …")
        gpw_lcao = relax_lcao(xyz_file)

    # --- Step 2: Ground-state FD calculation ---
    if os.path.exists(gpw_fd):
        print(f"✔ Cached FD found: {gpw_fd}")
    else:
        print(f"Running FD ground state for {prefix} …")
        gpw_fd = groundstate_fd(gpw_lcao)

    # --- Step 3: TDDFT spectrum ---
    if os.path.exists(spec_csv):
        print(f"✔ TDDFT already complete — skipping.")
        return spec_csv, spec_png
    else:
        sticks, spectrum = run_lrtddft(gpw_fd, prefix=prefix)
        print(f"TDDFT complete: {sticks}, {spectrum}")
        return sticks, spectrum


spectrum_files = run_pipeline("caf2_cluster.xyz")

# Run on Er-doped cluster
# spectrum_files = run_pipeline("caf2_er_cluster.xyz")


=== Starting pipeline for caf2_cluster ===
✔ Cached LCAO found: caf2_cluster_lcao.gpw
✔ Cached FD found: caf2_cluster_fd.gpw

=== Running LrTDDFT on caf2_cluster_fd.gpw ===
Available RAM ≈ 0.50 GB | KS states = 116

Kohn-Sham single transitions

KSS 2177 transitions (restrict={})
KSS TRK sum 16.3548 (15.838,16.5499,16.6765)
KSS polarisabilities(l=0-3) 16.3548, 3404.6, 2.44916e+07, 6.0256e+11

Linear response TDDFT calculation

RPA 2177 transitions
RPA kss[0]= # <KSSingle> 0->94 0(0) eji=43.7438[eV] (-0.000496737,-0.000496619,-0.000496593)
RPA estimated time left 17h54m58s
RPA kss[1]= # <KSSingle> 0->95 0(0) eji=43.8595[eV] (-7.32332e-05,-6.86547e-05,-6.8336e-05)
RPA estimated time left 14h51m11s
RPA kss[2]= # <KSSingle> 0->96 0(0) eji=43.9328[eV] (-1.89949e-06,-5.96262e-06,7.21282e-06)
RPA estimated time left 13h15m48s
RPA kss[3]= # <KSSingle> 0->97 0(0) eji=43.933[eV] (-9.57075e-06,5.03766e-06,3.87248e-06)
RPA estimated time left 12h57m35s
RPA kss[4]= # <KSSingle> 0->98 0(0) eji=43.9

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simpson
from scipy.ndimage import gaussian_filter1d
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def compute_optical_properties(sticks_csv, prefix="optical", broaden_sigma=0.1, atoms=None):
    """
    Convert TDDFT stick data to optical spectra with physical normalization
    and generate a quality plot (Energy ↔ Wavelength).

    Parameters
    ----------
    sticks_csv : str
        CSV file containing excitation energies (eV) and oscillator strengths.
    prefix : str
        Prefix for output filenames.
    broaden_sigma : float
        Gaussian broadening width in eV.
    atoms : ASE Atoms object, optional
        If provided, cell volume is extracted automatically (Å³ → m³).
    """

    # -----------------------------------------------------
    # Load TDDFT stick spectrum
    # -----------------------------------------------------
    data = np.loadtxt(sticks_csv, delimiter=",", skiprows=1)
    E, f = data[:, 0], data[:, 1]

    # Continuous energy grid
    x = np.linspace(0.001, max(E) + 2, 1500)  # start at 0.001 to avoid divide-by-zero
    y = np.zeros_like(x)
    for e_i, f_i in zip(E, f):
        idx = np.argmin(np.abs(x - e_i))
        y[idx] += f_i
    y = gaussian_filter1d(y, broaden_sigma * 100)

    # -----------------------------------------------------
    # Physical constants
    # -----------------------------------------------------
    eV_to_J = 1.602176634e-19
    hbar = 1.054571817e-34
    c = 2.99792458e8
    eps0 = 8.854187817e-12

    # -----------------------------------------------------
    # Get physical volume (m³)
    # -----------------------------------------------------
    if atoms is not None:
        volume = atoms.get_volume() * (1e-10)**3  # Å³ → m³
    else:
        volume = (10e-10)**3  # default ~1 nm³
    print(f"[INFO] Using normalization volume = {volume:.3e} m³")

    # -----------------------------------------------------
    # Dielectric response
    # -----------------------------------------------------
    eps2 = (2 * np.pi * eV_to_J * y) / (volume * eps0)

    eps1 = np.zeros_like(eps2)
    for i, Ei in enumerate(x):
        integrand = eps2 * x / (x**2 - Ei**2 + 1e-6)
        eps1[i] = 1 + (2 / np.pi) * simpson(integrand, x)

    # Complex refractive index
    n = np.sqrt(0.5 * (np.sqrt(eps1**2 + eps2**2) + eps1))
    kappa = np.sqrt(0.5 * (np.sqrt(eps1**2 + eps2**2) - eps1))

    # Absorption coefficient (cm⁻¹)
    alpha = (4 * np.pi * kappa * x * eV_to_J) / (hbar * c * 100)

    # -----------------------------------------------------
    # Save processed data
    # -----------------------------------------------------
    out_csv = f"{prefix}_optical_properties_scaled.csv"
    np.savetxt(
        out_csv,
        np.column_stack([x, eps1, eps2, n, kappa, alpha]),
        delimiter=",",
        header="Energy(eV),eps1,eps2,n,kappa,alpha(cm^-1)",
        comments=""
    )
    print(f"[DONE] Optical data saved: {out_csv}")

    # -----------------------------------------------------
    # Plot
    # -----------------------------------------------------
    mask = np.isfinite(x) & (x > 0)
    x_valid = x[mask]
    alpha_valid = alpha[mask]
    n_valid = n[mask]

    fig, ax1 = plt.subplots(figsize=(6, 4))
    ax1.plot(x_valid, alpha_valid / 1e5, color='tab:blue', label='Absorption (×1e5 cm⁻¹)')
    ax1.set_xlabel("Energy (eV)")
    ax1.set_ylabel("Absorption (a.u.)", color='tab:blue')

    # Twin axis for refractive index
    ax2 = ax1.twinx()
    ax2.plot(x_valid, n_valid, color='tab:orange', label='Refractive Index n(E)')
    ax2.set_ylabel("Refractive Index", color='tab:orange')

    # Top wavelength axis — safely limited
    def E_to_lambda(E): return np.clip(1240 / np.maximum(E, 1e-3), 100, 2000)
    def lambda_to_E(λ): return np.clip(1240 / np.maximum(λ, 1e-3), 0.1, 15)
    ax3 = ax1.secondary_xaxis('top', functions=(E_to_lambda, lambda_to_E))
    ax3.set_xlabel("Wavelength (nm)")

    # Highlight and annotate UV transition
    ax1.axvspan(7.5, 9.0, color='gray', alpha=0.15, label="UV absorption region")
    ax1.annotate("Main transition\n8.5 eV (146 nm)",
                 xy=(8.5, max(alpha_valid / 1e5) * 0.8),
                 xytext=(6, max(alpha_valid / 1e5) * 0.9),
                 arrowprops=dict(arrowstyle="->", color='black'),
                 fontsize=9)

    # Inset: low-energy polarization tail (2–4 eV)
    ax_inset = inset_axes(ax1, width="40%", height="35%", loc='lower right')
    ax_inset.plot(x_valid, alpha_valid / 1e5, color='tab:blue')
    ax_inset.set_xlim(2, 4)
    if np.any((x_valid > 2) & (x_valid < 4)):
        ax_inset.set_ylim(0, max(alpha_valid[(x_valid > 2) & (x_valid < 4)] / 1e5) * 1.2)
    ax_inset.set_title("Low-Energy Tail", fontsize=8)
    ax_inset.tick_params(axis='both', labelsize=7)

    # Layout
    fig.suptitle(f"{prefix} Optical Properties (Energy–Wavelength View)", fontsize=11)
    plt.subplots_adjust(top=0.85, bottom=0.15, right=0.88, left=0.15)

    # Safe saving (no NaN/Inf issues)
    plt.savefig(f"{prefix}_optical_caf2_cluster.png", dpi=500)
    plt.show()

    print(f"[CLEAN] Figure saved: {prefix}_optical_caf2_cluster.png")
    return out_csv, f"{prefix}_optical_caf2_cluster.png"


optical_files = compute_optical_properties("caf2_cluster_sticks.csv", prefix="caf2_cluster")